In [2]:
import pandas as pd
csv_cards = pd.read_csv("cards.csv", sep = "|")
csv_weather = pd.read_csv("weather.csv", sep = ";")

In [3]:
csv_cards.head(5)

,CP_CLIENTE,CP_COMERCIO,SECTOR,DIA,FRANJA_HORARIA,IMPORTE,NUM_OP
0,4007,4006,ALIMENTACION,2015-09-05,08-10,297.05,10.0
1,4007,4006,ALIMENTACION,2015-04-20,14-16,494.23,17.0
2,4007,4006,ALIMENTACION,2015-06-04,10-12,1694.15,57.0
3,4002,4009,ALIMENTACION,2015-12-05,20-22,79.71,2.0
4,4002,4007,ALIMENTACION,2015-03-07,20-22,181.28,3.0


In [4]:
csv_weather.head(5)

,FECHA,DIA,TMax,HTMax,TMin,HTMin,TMed,HumMax,HumMin,HumMed,VelViento,DirViento,Rad,Precip,ETo
0,2015-12-30,365,19.2,12:02,8.6,4:06,13.9,98.8,49.3,71.4,0.6,337.2,9.9,0.0,1.04
1,2015-12-29,363,18.8,12:38,10.4,4:32,14.2,100.0,64.7,81.2,1.0,292.7,8.5,0.0,1.07
2,2015-12-29,364,18.8,12:14,11.0,22:22,14.4,89.3,55.6,70.2,1.2,328.2,10.0,0.0,1.32
3,2015-12-28,362,18.9,10:00,11.6,23:48,15.8,89.0,47.4,66.1,1.2,280.8,9.3,0.0,1.38
4,2015-12-27,361,19.6,13:48,12.5,6:30,15.8,72.9,48.3,61.1,2.0,35.8,9.4,0.0,1.98


In [3]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 65.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=53ae8d2dd872c16e3215d17cf2eb8b9619eb3421fbe1960e9fe56c07a635d26b
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Dataframe').getOrCreate()
spark

In [5]:
## read the dataset
df_cards=spark.read.option('header','true').option('delimiter','|').csv('cards.csv',inferSchema=True)
df_cards.show(5)

+----------+-----------+------------+----------+--------------+-------+------+
|CP_CLIENTE|CP_COMERCIO|      SECTOR|       DIA|FRANJA_HORARIA|IMPORTE|NUM_OP|
+----------+-----------+------------+----------+--------------+-------+------+
|      4007|       4006|ALIMENTACION|2015-09-05|         08-10| 297.05|    10|
|      4007|       4006|ALIMENTACION|2015-04-20|         14-16| 494.23|    17|
|      4007|       4006|ALIMENTACION|2015-06-04|         10-12|1694.15|    57|
|      4002|       4009|ALIMENTACION|2015-12-05|         20-22|  79.71|     2|
|      4002|       4007|ALIMENTACION|2015-03-07|         20-22| 181.28|     3|
+----------+-----------+------------+----------+--------------+-------+------+
only showing top 5 rows



In [6]:
df_weather=spark.read.option('header','true').option('delimiter',';').csv('weather.csv',inferSchema=True)
df_weather.show(5)

+----------+---+----+-----+----+-----+----+------+------+------+---------+---------+----+------+----+
|     FECHA|DIA|TMax|HTMax|TMin|HTMin|TMed|HumMax|HumMin|HumMed|VelViento|DirViento| Rad|Precip| ETo|
+----------+---+----+-----+----+-----+----+------+------+------+---------+---------+----+------+----+
|2015-12-30|365|19.2|12:02| 8.6| 4:06|13.9|  98.8|  49.3|  71.4|      0.6|    337.2| 9.9|   0.0|1.04|
|2015-12-29|363|18.8|12:38|10.4| 4:32|14.2| 100.0|  64.7|  81.2|      1.0|    292.7| 8.5|   0.0|1.07|
|2015-12-29|364|18.8|12:14|11.0|22:22|14.4|  89.3|  55.6|  70.2|      1.2|    328.2|10.0|   0.0|1.32|
|2015-12-28|362|18.9|10:00|11.6|23:48|15.8|  89.0|  47.4|  66.1|      1.2|    280.8| 9.3|   0.0|1.38|
|2015-12-27|361|19.6|13:48|12.5| 6:30|15.8|  72.9|  48.3|  61.1|      2.0|     35.8| 9.4|   0.0|1.98|
+----------+---+----+-----+----+-----+----+------+------+------+---------+---------+----+------+----+
only showing top 5 rows



In [7]:
df_cards.printSchema()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df_cards.toPandas()

root
 |-- CP_CLIENTE: integer (nullable = true)
 |-- CP_COMERCIO: integer (nullable = true)
 |-- SECTOR: string (nullable = true)
 |-- DIA: string (nullable = true)
 |-- FRANJA_HORARIA: string (nullable = true)
 |-- IMPORTE: double (nullable = true)
 |-- NUM_OP: integer (nullable = true)



,CP_CLIENTE,CP_COMERCIO,SECTOR,DIA,FRANJA_HORARIA,IMPORTE,NUM_OP
0,4007,4006,ALIMENTACION,2015-09-05,08-10,297.05,10
1,4007,4006,ALIMENTACION,2015-04-20,14-16,494.23,17
2,4007,4006,ALIMENTACION,2015-06-04,10-12,1694.15,57
3,4002,4009,ALIMENTACION,2015-12-05,20-22,79.71,2
4,4002,4007,ALIMENTACION,2015-03-07,20-22,181.28,3
...,...,...,...,...,...,...,...
543126,4120,4001,OCIO Y TIEMPO LIBRE,2015-11-06,10-12,29.39,2
543127,4002,4009,OCIO Y TIEMPO LIBRE,2015-08-31,18-20,19.98,1
543128,4150,4001,OCIO Y TIEMPO LIBRE,2015-12-01,20-22,69.99,1
543129,4720,4001,OCIO Y TIEMPO LIBRE,2015-01-16,18-20,181.87,4


In [8]:
df_cards.createOrReplaceTempView("TCARDS")
df_weather.createOrReplaceTempView("TWEATHER")

**Propuestas de consultas:**
* Días de sol con gastos
* Media de gastos por sector
* Meses de gastos
* Horas de mayor gasto en verano
* Compras en día de lluvia
* Compras con temperaturas extremas

**Días de sol con gastos**

In [ ]:
dfgastosSun = spark.sql("")
dfgastosSun

**Media de gastos por sector**

In [9]:
dfsectores = spark.sql("SELECT SECTOR, AVG(IMPORTE) FROM TCARDS GROUP BY SECTOR;")
dfsectores

SECTOR,avg(IMPORTE)
HOGAR,84.73714679422375
AUTO,169.54961767800756
ALIMENTACION,152.45591312318547
MODA Y COMPLEMENTOS,132.51475219446198
BELLEZA,68.69483103448503
OCIO Y TIEMPO LIBRE,95.53525738460054


**Meses de gastos**

In [ ]:
dfgastosMonth = spark.sql("")
dfgastosMonth

**Horas de mayor gasto en verano**

In [ ]:
dfgastosSummer = spark.sql("")
dfgastosSummer

**Compras en día de lluvia**

In [ ]:
dfgastosRain = spark.sql("")
dfgastosRain

**Compras con temperaturas extremas**

In [ ]:
dfgastosXtrem = spark.sql("")
dfgastosXtrem

**LAZAMIENTO DE API**

In [10]:
pip install flask-ngrok

Esto lanza 3 direcciones obviamente no pilleis la local que nunca la va a pillar

In [14]:
from flask import Flask 
from flask_ngrok import run_with_ngrok 
from flask import Flask
from flask import jsonify
app = Flask(__name__) 
run_with_ngrok(app)    


@app.route("/") 
def home(): 
    return "<h1>Pruebas de llamada de la API</h1>"

@app.route('/api/v1/GastoSector', methods=['GET'])
def get_users():
    response = dfsectores
    return jsonify(response)

print("Lanzando servicio")
app.run()
print("Se ha acabado la ejecucion")

Se ha acabado la ejecucion
